# Unified TTS Notebook

**Single notebook for all TTS models and PDF extraction strategies**

This notebook provides a unified interface for:
- **TTS Models**: Kokoro (v0.9, v1.0), Maya1 (expressive, 20+ emotions), Silero v5
- **PDF Extractors**: Unstructured, PyMuPDF, Apple Vision, Nougat
- **Input Formats**: Text strings, PDF files, EPUB books
- **Output Formats**: WAV, MP3 with timeline manifests

The notebook will automatically install only the dependencies you need based on your selections!

✅ **Works both locally and in Google Colab** - automatically detects environment and downloads required files.

## 0a) Environment Detection & Setup

**This cell automatically detects if you're running in Google Colab or locally.**

If in Colab, it will download the required Python modules from the GitHub repository.

In [ ]:
import sys
import os
from pathlib import Path

# Detect if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("💻 Running locally")

# GitHub repository URL for downloading Python modules
GITHUB_RAW_URL = "https://raw.githubusercontent.com/SVM0N/ttsweb.github.io/main/"

# Required Python modules (in tts_lib folder)
REQUIRED_MODULES = [
    "tts_lib/__init__.py",
    "tts_lib/config.py",
    "tts_lib/tts_backends.py",
    "tts_lib/tts_utils.py",
    "tts_lib/pdf_extractors.py",
    "tts_lib/manifest.py",
    "tts_lib/setup.py",
    "tts_lib/init_system.py",
    "tts_lib/synthesis.py"
]

if IN_COLAB:
    print("\n📦 Setting up Colab environment...")
    print("   Downloading required Python modules from GitHub...")
    
    import urllib.request
    
    # Create tts_lib directory
    Path("tts_lib").mkdir(exist_ok=True)
    
    for module in REQUIRED_MODULES:
        url = GITHUB_RAW_URL + module
        try:
            print(f"   → Downloading {module}...")
            urllib.request.urlretrieve(url, module)
            print(f"   ✓ {module} downloaded")
        except Exception as e:
            print(f"   ✗ Failed to download {module}: {e}")
            print(f"     URL: {url}")
    
    # Create files directory for outputs
    files_dir = Path("files")
    files_dir.mkdir(exist_ok=True)
    print(f"\n✓ Created output directory: {files_dir}")
    
    print("\n✓ Colab environment setup complete!")
    print("  You can now proceed with the rest of the notebook.")
    print("\n📝 Note: To upload PDFs or EPUBs, use the file upload button in the sidebar")
    print("  or run: from google.colab import files; uploaded = files.upload()")
    
else:
    print("\n✓ Local environment detected")
    print("  Using local Python modules")
    
    # Check if required modules exist locally
    missing_modules = []
    for module in REQUIRED_MODULES:
        if not Path(module).exists():
            missing_modules.append(module)
    
    if missing_modules:
        print(f"\n⚠️  Warning: Missing modules: {', '.join(missing_modules)}")
        print("  Make sure you're running this notebook from the repository directory")
    else:
        print(f"  ✓ All required modules found")

print("\n" + "="*60)

## 0b) Conda Environment Setup (Optional - Local Only)

**This step helps you manage Python packages and avoid conflicts with your system installation.**

- If you have **conda** installed, you can create a fresh environment for this notebook
- Or use an existing environment by providing its name
- At the end of the notebook, you can easily clean up and delete the environment to free storage
- **Note**: This section is only relevant for local installations, not Google Colab

In [ ]:
import subprocess
import sys
import os

# Flag to track if we created an environment in this notebook
environment_created_by_notebook = False
environment_name = None

# Check if conda is installed
try:
    result = subprocess.run(['conda', '--version'], capture_output=True, text=True, check=True)
    conda_available = True
    print(f"✓ Conda detected: {result.stdout.strip()}")
except (subprocess.CalledProcessError, FileNotFoundError):
    conda_available = False
    print("✗ Conda not found - skipping environment management")
    print("Packages will be installed in your current Python environment")

if conda_available:
    print("\n" + "="*60)
    print("ENVIRONMENT SETUP OPTIONS")
    print("="*60)
    
    choice = input("\nDo you want to:\n  [1] Create a NEW conda environment (recommended)\n  [2] Use an EXISTING environment\n  [3] Skip and use current environment\n\nEnter choice (1/2/3): ").strip()
    
    if choice == "1":
        env_name = input("\nEnter name for new environment (default: tts_unified): ").strip()
        if not env_name:
            env_name = "tts_unified"
        
        print(f"\n→ Creating conda environment: {env_name}")
        print("  This may take a few minutes...")
        
        try:
            subprocess.run(['conda', 'create', '-n', env_name, 'python=3.10', '-y'],
                           check=True, capture_output=True)
            
            environment_created_by_notebook = True
            environment_name = env_name
            
            print(f"✓ Environment '{env_name}' created successfully!")
            print(f"\n{'='*60}")
            print("IMPORTANT: Restart your Jupyter kernel and select the new environment:")
            print(f"  Kernel → Change Kernel → {env_name}")
            print(f"{'='*60}\n")
            
        except subprocess.CalledProcessError as e:
            print(f"✗ Failed to create environment: {e}")
            print("Continuing with current environment...")
    
    elif choice == "2":
        env_name = input("\nEnter name of existing environment: ").strip()
        if env_name:
            environment_name = env_name
            print(f"\n✓ Using existing environment: {env_name}")
            print(f"\n{'='*60}")
            print("IMPORTANT: Make sure your kernel is using this environment:")
            print(f"  Kernel → Change Kernel → {env_name}")
            print(f"{'='*60}\n")
        else:
            print("✗ No environment name provided - using current environment")
    
    else:
        print("\n✓ Using current environment")

print("\nYou can now proceed with the rest of the notebook.")

## 1) Configuration - Choose Your Setup

**Select which TTS model, PDF extractor, and formats you want to use.**

The notebook will automatically install only the dependencies you need!

In [ ]:
# ========================================
# TTS MODEL SELECTION
# ========================================
# Choose ONE of the following:
#   - "kokoro_0.9": Kokoro v0.9+ (10 voices, English-focused, stable)
#   - "kokoro_1.0": Kokoro v1.0 (54 voices, 8 languages, latest)
#   - "maya1": Maya1 (20+ emotions, natural language voices, expressive, requires GPU)
#   - "silero_v5": Silero v5 (Russian language, 6 speakers)

TTS_MODEL = "kokoro_1.0"

# ========================================
# PDF EXTRACTOR SELECTION
# ========================================
# Choose ONE of the following:
#   - "unstructured": Advanced layout analysis (recommended, ~500MB dependencies)
#   - "pymupdf": Fast extraction for clean PDFs (~15MB, lightweight)
#   - "vision": OCR for scanned PDFs (macOS only)
#   - "nougat": Academic papers with equations (~1.5GB model)
#   - None: Skip PDF extraction (only for text/EPUB input)

PDF_EXTRACTOR = "unstructured"

# ========================================
# INPUT FORMATS
# ========================================
# Enable the input formats you plan to use:

ENABLE_TEXT_INPUT = False    # Plain text strings
ENABLE_PDF_INPUT = True     # PDF files (requires PDF_EXTRACTOR)
ENABLE_EPUB_INPUT = False    # EPUB books

# ========================================
# OUTPUT FORMATS
# ========================================
# Enable the output formats you plan to use:

ENABLE_WAV_OUTPUT = False    # WAV audio files
ENABLE_MP3_OUTPUT = True    # MP3 audio files (requires ffmpeg and pydub)

# ========================================
# DEVICE CONFIGURATION
# ========================================
# Device to use for TTS synthesis:
#   - "auto": Automatically select best device (CUDA > MPS > CPU)
#   - "cuda": Force CUDA/GPU (required for Maya1)
#   - "cpu": Force CPU
#   - "mps": Force Apple Silicon MPS

DEVICE = "auto"

# ========================================
# OUTPUT DIRECTORY
# ========================================
# Directory where generated files will be saved

OUTPUT_DIR = "files"  # Files directory for PDFs and outputs

# ========================================
# VALIDATION
# ========================================
if ENABLE_PDF_INPUT and PDF_EXTRACTOR is None:
    print("⚠️  WARNING: PDF input enabled but no PDF extractor selected!")
    print("   Set PDF_EXTRACTOR to 'unstructured', 'pymupdf', 'vision', or 'nougat'")

if TTS_MODEL == "maya1":
    import torch
    if not torch.cuda.is_available():
        print("⚠️  WARNING: Maya1 requires CUDA GPU!")
        print("   Maya1 will not work properly on CPU or MPS.")
        print("   Consider using Kokoro or Silero models instead.")

print("="*60)
print("CONFIGURATION SUMMARY")
print("="*60)
print(f"TTS Model: {TTS_MODEL}")
print(f"PDF Extractor: {PDF_EXTRACTOR or 'None'}")
print(f"Input Formats: Text={ENABLE_TEXT_INPUT}, PDF={ENABLE_PDF_INPUT}, EPUB={ENABLE_EPUB_INPUT}")
print(f"Output Formats: WAV={ENABLE_WAV_OUTPUT}, MP3={ENABLE_MP3_OUTPUT}")
print(f"Device: {DEVICE}")
print(f"Output Directory: {OUTPUT_DIR}")
print("="*60)

## 1.5) Apple Silicon (MPS) Fix

**Automatically detect and fix Apple Silicon compatibility issues.**

If you're on Apple Silicon, this will enable CPU fallback for unsupported operations.

In [ ]:
import os
import platform

# Check if we're on macOS with Apple Silicon
is_apple_silicon = (
    platform.system() == "Darwin" and 
    platform.machine() == "arm64"
)

if is_apple_silicon:
    print("🍎 Apple Silicon detected")
    print("   Enabling MPS fallback for unsupported operations...")
    
    # Set environment variable to enable CPU fallback for unsupported MPS operations
    # This fixes the 'aten::angle not implemented for MPS' error
    os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
    
    print("   ✓ MPS fallback enabled")
    print("   Note: Some operations will fall back to CPU (slightly slower but works)")
else:
    print("✓ No Apple Silicon-specific fixes needed")

## 2) Install Dependencies

**Running automatic dependency installation...**

This will install only what you need based on your configuration.

In [ ]:
from tts_lib.setup import install_dependencies

# Install dependencies based on configuration
install_dependencies(
    tts_model=TTS_MODEL,
    pdf_extractor=PDF_EXTRACTOR,
    enable_pdf_input=ENABLE_PDF_INPUT,
    enable_epub_input=ENABLE_EPUB_INPUT,
    enable_mp3_output=ENABLE_MP3_OUTPUT
)

print("\n🚀 Ready to initialize system!")

## 3) Initialize TTS System

**Loading TTS model and PDF extractor...**

In [ ]:
from tts_lib.init_system import initialize_system

# Initialize TTS backend, config, and PDF extractor
tts, config, pdf_extractor = initialize_system(
    tts_model=TTS_MODEL,
    output_dir=OUTPUT_DIR,
    device=DEVICE,
    pdf_extractor_name=PDF_EXTRACTOR,
    enable_pdf_input=ENABLE_PDF_INPUT
)

## 4) Ready to Synthesize!

All synthesis functions are loaded from the `synthesis.py` module.

You can now use:
- `synth_string()` - Convert text to audio
- `synth_pdf()` - Convert PDF to audio  
- `synth_epub()` - Convert EPUB to per-chapter audio ZIP

Just run the example cells below!

In [ ]:
# Import synthesis functions from tts_lib
from tts_lib.synthesis import synth_string, synth_pdf, synth_epub

print("✓ Synthesis functions ready:")
print("  • synth_string(text, voice, speed, format, ...)")
print("  • synth_pdf(pdf_path, voice, speed, format, pages, ...)")
print("  • synth_epub(epub_path, voice, speed, format, ...)")
print("\n💡 See example cells below for usage")

## 4.5) File Upload Helper (Colab Only)

**If you're in Google Colab and want to upload PDFs or EPUBs, run this cell.**

This provides an easy way to upload files to the Colab environment.

In [ ]:
if IN_COLAB:
    print("📤 File Upload for Google Colab")
    print("="*60)
    print("Run this cell to upload PDF or EPUB files to Colab")
    print()
    
    from google.colab import files
    import shutil
    
    # Upload files
    uploaded = files.upload()
    
    # Move uploaded files to 'files' directory
    for filename in uploaded.keys():
        dest_path = f"files/{filename}"
        shutil.move(filename, dest_path)
        print(f"✓ Moved {filename} to {dest_path}")
    
    print()
    print("✓ Upload complete! You can now use these files in the examples below.")
    print(f"  Uploaded files: {list(uploaded.keys())}")
else:
    print("⚠️  This cell is only for Google Colab")
    print("  You're running locally, so use your file system directly")

## Usage Examples

Run the examples below to synthesize text, PDFs, and EPUBs.

**Note:** Only the examples for enabled input/output formats will work.

### A) String → Audio

In [ ]:
if not ENABLE_TEXT_INPUT:
    print("⚠️  Text input is disabled. Set ENABLE_TEXT_INPUT=True to use this example.")
else:
    # Configuration
    VOICE = None  # Use default voice (or specify a voice description)
    SPEED = 1.0   # Speech speed (Kokoro and Maya1 only)
    FORMAT = "mp3" if ENABLE_MP3_OUTPUT else "wav"
    BASENAME = "tts_text"

    # Text to synthesize
    # For Maya1: You can add emotion tags like <laugh>, <whisper>, <cry>, etc.
    TEXT = """Hello! This is a test of the unified TTS system.
    It automatically installs only the dependencies you need.
    """

    # Run synthesis
    audio_path, manifest_path = synth_string(
        tts=tts,
        config=config,
        text=TEXT,
        voice=VOICE,
        speed=SPEED,
        out_format=FORMAT,
        basename=BASENAME,
        tts_model=TTS_MODEL,
        enable_text_input=ENABLE_TEXT_INPUT,
        enable_mp3_output=ENABLE_MP3_OUTPUT
    )

    print(f"\n✓ Audio saved to: {audio_path}")
    print(f"✓ Manifest saved to: {manifest_path}")
    
    if IN_COLAB:
        print("\n💡 To download the files, run:")
        print(f"   from google.colab import files")
        print(f"   files.download('{audio_path}')")
        print(f"   files.download('{manifest_path}')")

### B) PDF → Audio (with page selection)

In [ ]:
if not ENABLE_PDF_INPUT:
    print("⚠️  PDF input is disabled. Set ENABLE_PDF_INPUT=True and PDF_EXTRACTOR to use this example.")
else:
    # Configuration
    VOICE = None  # Use default voice
    SPEED = 1.0
    FORMAT = "mp3" if ENABLE_MP3_OUTPUT else "wav"

    # PDF file path
    # For Google Colab: Upload your PDF first using the file upload button
    # For local: Use the path to your PDF file
    PDF_PATH = "files/Case1Writeup.pdf"  # Change this to your PDF filename
    
    # Check if file exists, provide helpful message if not
    import os
    if not os.path.exists(PDF_PATH):
        print(f"⚠️  PDF file not found: {PDF_PATH}")
        if IN_COLAB:
            print("\n📤 To upload a PDF in Colab, run:")
            print("   from google.colab import files")
            print("   uploaded = files.upload()")
            print("   # Then move it: !mv uploaded_file.pdf files/")
        else:
            print("\n💡 Make sure your PDF is in the 'files' directory")
            print("   Or update PDF_PATH to point to your PDF file")
    else:
        # Page selection (optional)
        # None = all pages (default)
        # [1, 2, 3] = only pages 1, 2, and 3
        # [5] = only page 5
        PAGES = None

        # Run synthesis
        audio_path, manifest_path = synth_pdf(
            tts=tts,
            config=config,
            pdf_extractor=pdf_extractor,
            file_path_or_bytes=PDF_PATH,
            voice=VOICE,
            speed=SPEED,
            out_format=FORMAT,
            pages=PAGES,
            tts_model=TTS_MODEL,
            enable_pdf_input=ENABLE_PDF_INPUT,
            enable_mp3_output=ENABLE_MP3_OUTPUT
        )

        print(f"\n✓ Audio saved to: {audio_path}")
        print(f"✓ Manifest saved to: {manifest_path}")
        
        if IN_COLAB:
            print("\n💡 To download the files, run:")
            print(f"   from google.colab import files")
            print(f"   files.download('{audio_path}')")
            print(f"   files.download('{manifest_path}')")

### C) EPUB → ZIP (Per-Chapter Audio)

In [ ]:
if not ENABLE_EPUB_INPUT:
    print("⚠️  EPUB input is disabled. Set ENABLE_EPUB_INPUT=True to use this example.")
else:
    # Configuration
    VOICE = None  # Use default voice
    SPEED = 1.0
    CHAPTER_FORMAT = "mp3" if ENABLE_MP3_OUTPUT else "wav"
    ZIP_NAME = ""  # Optional: custom name for ZIP file

    # EPUB file path
    # For Google Colab: Upload your EPUB first
    # For local: Use the path to your EPUB file
    EPUB_PATH = "book.epub"  # Change this to your EPUB filename
    
    # Check if file exists, provide helpful message if not
    import os
    if not os.path.exists(EPUB_PATH):
        print(f"⚠️  EPUB file not found: {EPUB_PATH}")
        if IN_COLAB:
            print("\n📤 To upload an EPUB in Colab, run:")
            print("   from google.colab import files")
            print("   uploaded = files.upload()")
        else:
            print("\n💡 Make sure your EPUB file exists")
            print("   Or update EPUB_PATH to point to your EPUB file")
    else:
        # Run synthesis
        zip_path = synth_epub(
            tts=tts,
            config=config,
            file_path_or_bytes=EPUB_PATH,
            voice=VOICE,
            speed=SPEED,
            per_chapter_format=CHAPTER_FORMAT,
            zip_name=(ZIP_NAME or None),
            tts_model=TTS_MODEL,
            enable_epub_input=ENABLE_EPUB_INPUT,
            enable_mp3_output=ENABLE_MP3_OUTPUT
        )

        print(f"\n✓ ZIP archive saved to: {zip_path}")
        
        if IN_COLAB:
            print("\n💡 To download the ZIP file, run:")
            print(f"   from google.colab import files")
            print(f"   files.download('{zip_path}')")

## Notes

- **Switching Models**: To use a different TTS model or PDF extractor, change the settings in Section 1 and re-run from there
- **Voice Selection**: Each model has different voices. Check the output of Section 3 for available voices
- **Manifest Files**: Each audio output includes a JSON manifest with sentence-level timing and coordinates
- **Dependencies**: Only the packages needed for your selected configuration were installed

## Cleanup: Delete Environment (Optional)

**If you created a new environment at the beginning of this notebook**, you can delete it here to free up storage space.

⚠️ **Warning**: This will permanently delete the environment and all installed packages!

In [ ]:
import subprocess

if 'environment_created_by_notebook' not in globals():
    print("✗ No environment tracking found")
    print("This cell only works if you ran the environment setup cell at the beginning")
elif not environment_created_by_notebook:
    print("✗ No environment was created by this notebook")
    print("You can only delete environments that were created in this session")
else:
    print(f"Environment '{environment_name}' was created by this notebook")
    print(f"\n{'='*60}")
    print("DELETE ENVIRONMENT")
    print(f"{'='*60}")
    
    confirm = input(f"\nAre you sure you want to DELETE '{environment_name}'?\nType 'yes' to confirm: ").strip().lower()
    
    if confirm == 'yes':
        print(f"\n→ Deleting environment '{environment_name}'...")
        print("  This may take a moment...")
        
        try:
            subprocess.run(['conda', 'env', 'remove', '-n', environment_name, '-y'],
                           check=True, capture_output=True)
            print(f"✓ Environment '{environment_name}' deleted successfully!")
            print("  Storage space has been freed.")
            
            environment_created_by_notebook = False
            environment_name = None
            
        except subprocess.CalledProcessError as e:
            print(f"✗ Failed to delete environment: {e}")
            print(f"You may need to delete it manually with: conda env remove -n {environment_name}")
    else:
        print("\n✗ Deletion cancelled - environment preserved")

## Memory Management: View & Delete Models

**View all locally cached models, their sizes, and manage storage.**

This section helps you:
- See which models are downloaded and how much space they use
- Delete specific models to free up storage
- Clean PyTorch and HuggingFace caches
- Clear GPU/MPS memory

In [ ]:
import os
import shutil
from pathlib import Path

def get_dir_size(path):
    """Calculate total size of a directory in bytes."""
    total = 0
    try:
        for entry in os.scandir(path):
            if entry.is_file(follow_symlinks=False):
                total += entry.stat().st_size
            elif entry.is_dir(follow_symlinks=False):
                total += get_dir_size(entry.path)
    except (PermissionError, FileNotFoundError):
        pass
    return total

def format_bytes(bytes_size):
    """Format bytes to human-readable size."""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024.0:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024.0
    return f"{bytes_size:.2f} PB"

def scan_cached_models():
    """Scan for cached models in common locations."""
    home = Path.home()
    
    cache_locations = {
        "HuggingFace Cache": home / ".cache" / "huggingface",
        "Torch Hub": home / ".cache" / "torch" / "hub",
        "Torch Checkpoints": home / ".cache" / "torch" / "checkpoints",
        "Detectron2": home / ".torch" / "fvcore_cache" / "detectron2",
        "Kokoro Models": home / ".cache" / "kokoro",
    }
    
    results = []
    total_size = 0
    
    print("=" * 80)
    print("SCANNING CACHED MODELS")
    print("=" * 80)
    print()
    
    for name, path in cache_locations.items():
        if path.exists():
            size = get_dir_size(path)
            if size > 0:
                results.append({
                    "name": name,
                    "path": str(path),
                    "size": size,
                    "size_formatted": format_bytes(size)
                })
                total_size += size
                print(f"📦 {name}")
                print(f"   Path: {path}")
                print(f"   Size: {format_bytes(size)}")
                
                # Try to list specific models if it's HuggingFace cache
                if name == "HuggingFace Cache":
                    models_dir = path / "hub"
                    if models_dir.exists():
                        model_folders = [d for d in models_dir.iterdir() if d.is_dir() and d.name.startswith("models--")]
                        if model_folders:
                            print(f"   Models found: {len(model_folders)}")
                            for model_folder in sorted(model_folders)[:5]:  # Show first 5
                                model_name = model_folder.name.replace("models--", "").replace("--", "/")
                                model_size = get_dir_size(model_folder)
                                print(f"     • {model_name}: {format_bytes(model_size)}")
                            if len(model_folders) > 5:
                                print(f"     ... and {len(model_folders) - 5} more")
                
                # Try to list PyTorch Hub models
                elif name == "Torch Hub":
                    checkpoints = list(path.glob("checkpoints/*.pth")) + list(path.glob("*.pth"))
                    if checkpoints:
                        print(f"   Checkpoints found: {len(checkpoints)}")
                        for ckpt in sorted(checkpoints)[:5]:
                            ckpt_size = ckpt.stat().st_size
                            print(f"     • {ckpt.name}: {format_bytes(ckpt_size)}")
                        if len(checkpoints) > 5:
                            print(f"     ... and {len(checkpoints) - 5} more")
                
                print()
        else:
            print(f"⚪ {name}")
            print(f"   Path: {path}")
            print(f"   Status: Not found")
            print()
    
    print("=" * 80)
    print(f"TOTAL CACHED SIZE: {format_bytes(total_size)}")
    print("=" * 80)
    
    return results, total_size

# Run the scan
cached_models, total_cache_size = scan_cached_models()

### Delete Cached Models

**Select which caches to delete.**

Choose what you want to clean up to free storage space.

In [ ]:
def delete_cache(cache_name):
    """Delete a specific cache directory."""
    home = Path.home()
    
    cache_paths = {
        "huggingface": home / ".cache" / "huggingface",
        "torch_hub": home / ".cache" / "torch" / "hub",
        "torch_checkpoints": home / ".cache" / "torch" / "checkpoints",
        "detectron2": home / ".torch" / "fvcore_cache" / "detectron2",
        "kokoro": home / ".cache" / "kokoro",
        "pip": home / ".cache" / "pip",
    }
    
    if cache_name not in cache_paths:
        print(f"✗ Unknown cache: {cache_name}")
        print(f"Available caches: {', '.join(cache_paths.keys())}")
        return False
    
    cache_path = cache_paths[cache_name]
    
    if not cache_path.exists():
        print(f"⚪ Cache not found: {cache_path}")
        return False
    
    size_before = get_dir_size(cache_path)
    print(f"→ Deleting {cache_name} cache...")
    print(f"  Path: {cache_path}")
    print(f"  Size: {format_bytes(size_before)}")
    
    try:
        shutil.rmtree(cache_path)
        print(f"✓ Successfully deleted {cache_name} cache")
        print(f"  Freed: {format_bytes(size_before)}")
        return True
    except Exception as e:
        print(f"✗ Failed to delete cache: {e}")
        return False


# Interactive deletion
print("=" * 80)
print("DELETE CACHE OPTIONS")
print("=" * 80)
print()
print("Available caches to delete:")
print("  [1] HuggingFace Cache")
print("  [2] Torch Hub")
print("  [3] Torch Checkpoints")
print("  [4] Detectron2")
print("  [5] Kokoro Models")
print("  [6] Pip Cache")
print("  [7] ALL caches (⚠️  WARNING: Deletes everything!)")
print("  [0] Cancel")
print()

choice = input("Enter choice (0-7): ").strip()

cache_map = {
    "1": "huggingface",
    "2": "torch_hub",
    "3": "torch_checkpoints",
    "4": "detectron2",
    "5": "kokoro",
    "6": "pip",
}

if choice == "0":
    print("\n✗ Deletion cancelled")
elif choice == "7":
    confirm = input("\n⚠️  Delete ALL caches? Type 'yes' to confirm: ").strip().lower()
    if confirm == "yes":
        print("\n→ Deleting all caches...")
        total_freed = 0
        for cache_name in cache_map.values():
            if delete_cache(cache_name):
                print()
        print("=" * 80)
        print("✓ All caches deleted")
        print("=" * 80)
    else:
        print("\n✗ Deletion cancelled")
elif choice in cache_map:
    cache_name = cache_map[choice]
    confirm = input(f"\nDelete {cache_name} cache? Type 'yes' to confirm: ").strip().lower()
    if confirm == "yes":
        print()
        delete_cache(cache_name)
    else:
        print("\n✗ Deletion cancelled")
else:
    print("\n✗ Invalid choice")

### Clear GPU/MPS Memory

**Free up GPU or MPS (Apple Silicon) memory.**

Run this cell to clear PyTorch's memory cache and force garbage collection.

In [ ]:
import gc
import torch

print("=" * 80)
print("CLEARING GPU/MPS MEMORY")
print("=" * 80)
print()

# Check current memory usage (if applicable)
if torch.cuda.is_available():
    print("📊 CUDA Memory Status (before cleanup):")
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**2
        reserved = torch.cuda.memory_reserved(i) / 1024**2
        print(f"   GPU {i}: {allocated:.2f} MB allocated, {reserved:.2f} MB reserved")
    print()
elif torch.backends.mps.is_available():
    print("📊 MPS (Apple Silicon) detected")
    print("   Note: MPS doesn't provide detailed memory stats")
    print()
else:
    print("📊 No GPU/MPS detected - running on CPU")
    print()

# Clear PyTorch cache
print("→ Clearing PyTorch cache...")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print("✓ CUDA cache cleared")
elif torch.backends.mps.is_available():
    torch.mps.empty_cache()
    torch.mps.synchronize()
    print("✓ MPS cache cleared")
else:
    print("⚪ No GPU cache to clear (CPU mode)")

# Force garbage collection
print("\n→ Running garbage collection...")
gc.collect()
print("✓ Garbage collection completed")

# Check memory usage after cleanup
print()
if torch.cuda.is_available():
    print("📊 CUDA Memory Status (after cleanup):")
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**2
        reserved = torch.cuda.memory_reserved(i) / 1024**2
        print(f"   GPU {i}: {allocated:.2f} MB allocated, {reserved:.2f} MB reserved")
elif torch.backends.mps.is_available():
    print("📊 MPS cache has been cleared")
    print("   Memory should be freed for other applications")

print()
print("=" * 80)
print("✓ MEMORY CLEANUP COMPLETE")
print("=" * 80)